In [ ]:
from espspy.readers import EspsFormantReader
import os, sys, fnmatch
from audiolabel import read_label
import re
import pandas as pd
from sys import argv
from phonlab.utils import dir2df
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Get textgrid information

In [1]:
tgdir = ('/home/ubuntu/Desktop/Shared/sf_Box_Sync/Diss_data/Fall_2019_LRAP/Word_lists/2018_vowels/2018_data')

In [ ]:
fnpat = '^(?P<spkr>[^_]+)_(?P<age_yrs>\d+|adult)_(?P<task>.+)\.TextGrid$' # filenames

tgdf = dir2df(
    tgdir,
    fnpat=fnpat,
    addcols=['dirname', 'barename', 'ext']   # addcols spits out relevant file names
)
tgdf.head()

In [ ]:
# only get children aged 4-6
tgdf = tgdf[
    tgdf.age_yrs.isin(['4', '5', '6']) 
]

In [ ]:
dflist = []
for row in tgdf.itertuples():
    print(row.dirname+row.fname)
    [phdf, wddf] = read_label(
        os.path.join(row.dirname,row.fname), 
        'praat', tiers = ['Phone','Word'], 
        addcols=['dirname', 'barename', 'ext']
    )   
                             
    dflist.append(pd.merge_asof(phdf, wddf, on='t1', suffixes=['_ph', '_wd']))

In [ ]:
phwddf = pd.concat(dflist) # one df for all kids with words and phones

In [ ]:
dffinal = phwddf.merge(
    tgdf,
    left_on=['barename_ph'], 
    right_on=['barename'],
    how='outer'
)

In [ ]:
# subset to create df with just relevant vowels
voweldf = dffinal[
    dffinal.Phone.isin(['a', 'e', 'i', 'o', 'u']) 
]

In [ ]:
# remove tokens under 50ms
voweldf2 = voweldf[voweldf.t2_ph - voweldf.t1 >= .05] 

In [ ]:
# subset only the relevant children (aged 4-6)
subchild = voweldf2[voweldf2.age_yrs.isin(['4', '5', '6'])]

In [ ]:
# this function matches tg rows with nearest timestamp in covrdrdf, acdf, or ifcdf,  
# and is required for the following functions
# current version only tracks F1 and F2 as these are the only formants
# tracked when lpc order=8

def get_row_nearest_time(df, t, timecol): # timecol = timestamp col in df
    '''Get the row nearest in time to specified value.
    In case of tie, first row is returned.
    ***NOTE***
    df[timecol] must be sorted!!!
    '''
    return pd.DataFrame.from_records([df.loc[(df[timecol] - t).abs().idxmin()]])

## Process "formant" .cov results

In [ ]:
# define function to match .cov meas and tg info

def formant_cov_summary(row):
    phone_dur = row.t2_ph - row.t1
    midpt = (phone_dur*.5)+row.t1 
    qrtr = (phone_dur*.25)+row.t1 
    thrqrtr = (phone_dur*.75)+row.t1 

    covdir = ('/home/ubuntu/Desktop/Shared/sf_Box_Sync/Diss_data/Fall_2019_LRAP/Word_lists/2018_vowels/2018_data/formants_cov_lpcorder_8')

    covdr = EspsFormantReader(os.path.join(covdir, row.barename + '.fb')) # make an ESPS object of the associated .fb file
    covdrdf = covdr.get_df(t1=row.t1, t2=row.t2_ph) # 

    midptrow = get_row_nearest_time(covdrdf, midpt, 'times')
    midptdf = midptrow.rename(columns={"times": "midpt_sec", "fm1": "midpt_f1", "fm2": "midpt_f2"})
    qrtrrow = get_row_nearest_time(covdrdf, qrtr, 'times')
    qrtrdf = qrtrrow.rename(columns={"times": "qrtr_sec", "fm1": "qrtr_f1", "fm2": "qrtr_f2"})
    thrqrtrrow = get_row_nearest_time(covdrdf, thrqrtr, 'times')
    thrqrtrdf = thrqrtrrow.rename(columns={"times": "thrqrtr_sec", "fm1": "thrqrtr_f1", "fm2": "thrqrtr_f2"})

    rowdf = pd.DataFrame([row])

    rowdffinal = rowdf.reindex(columns=['t1', 't2_ph', 'Phone', 'barename', 'Word', 'spkr', 'age_yrs', 'task']) # :=all rows, and these columns

    return pd.concat(
        [
            midptdf.reindex(columns=['midpt_sec', 'midpt_f1', 'midpt_f2']), # subset relevant cols
            qrtrdf.reindex(columns=['qrtr_sec', 'qrtr_f1', 'qrtr_f2']),
            thrqrtrdf.reindex(columns=['thrqrtr_sec', 'thrqrtr_f1', 'thrqrtr_f2']),
            rowdffinal # plus cols w/ metadata
        ], 
        axis='columns' # add cols, not rows
    )

In [ ]:
# now run the function to match formant meas and tg info

covdflist = []

for row in subchild.itertuples(): 
    covdflist.append(formant_cov_summary(row)) 

cov_final = pd.concat(covdflist)

## Process "formant" .ac results

In [ ]:
# define function to match .ac meas and tg info

def formant_ac_summary(row):
    phone_dur = row.t2_ph - row.t1
    midpt = (phone_dur*.5)+row.t1 
    qrtr = (phone_dur*.25)+row.t1 
    thrqrtr = (phone_dur*.75)+row.t1 

    acdir = ('/home/ubuntu/Desktop/Shared/sf_Box_Sync/Diss_data/Fall_2019_LRAP/Word_lists/2018_vowels/2018_data/formants_ac_lpcorder_8')

    acrdr = EspsFormantReader(os.path.join(acdir, row.barename + '.fb')) # make an ESPS object of the associated .fb file
    acrdrdf = acrdr.get_df(t1=row.t1, t2=row.t2_ph) # 

    midptrow = get_row_nearest_time(acrdrdf, midpt, 'times')
    midptdf = midptrow.rename(columns={"times": "midpt_sec", "fm1": "midpt_f1", "fm2": "midpt_f2"})
    qrtrrow = get_row_nearest_time(acrdrdf, qrtr, 'times')
    qrtrdf = qrtrrow.rename(columns={"times": "qrtr_sec", "fm1": "qrtr_f1", "fm2": "qrtr_f2"})
    thrqrtrrow = get_row_nearest_time(acrdrdf, thrqrtr, 'times')
    thrqrtrdf = thrqrtrrow.rename(columns={"times": "thrqrtr_sec", "fm1": "thrqrtr_f1", "fm2": "thrqrtr_f2"})

    rowdf = pd.DataFrame([row])

    rowdffinal = rowdf.reindex(columns=['t1', 't2_ph', 'Phone', 'barename', 'Word', 'spkr', 'age_yrs', 'task']) # :=all rows, and these columns

    return pd.concat(
        [
            midptdf.reindex(columns=['midpt_sec', 'midpt_f1', 'midpt_f2']), # subset relevant cols
            qrtrdf.reindex(columns=['qrtr_sec', 'qrtr_f1', 'qrtr_f2']),
            thrqrtrdf.reindex(columns=['thrqrtr_sec', 'thrqrtr_f1', 'thrqrtr_f2']),
            rowdffinal # plus cols w/ metadata
        ], 
        axis='columns' # add cols, not rows
    )

In [ ]:
# now run the function to match formant meas and tg info

acdflist = []

for row in voweldf2.itertuples(): 
    acdflist.append(formant_ac_summary(row)) 

ac_final = pd.concat(acdflist)

## Process ifc_formant results

In [ ]:
# define function to match meas and tg info

def ifc_summary(row):
    
    phone_dur = row.t2_ph - row.t1
    midpt = (phone_dur*.5)+row.t1 
    qrtr = (phone_dur*.25)+row.t1 
    thrqrtr = (phone_dur*.75)+row.t1 
    
    ifcdir = ('/home/ubuntu/Desktop/Shared/sf_Box_Sync/Diss_data/Fall_2019_LRAP/Word_lists/2018_vowels/2018_data/formants_ifc')

    ifcname = row.barename + '.ifc' # get ifc filename (just a string)
    df = pd.read_table(os.path.join(ifcdir, ifcname)) # big df of ifc values

    midptrow = get_row_nearest_time(df, midpt, 'sec')
    midptdf = midptrow.rename(columns={"sec": "midpt_sec", "f1": "midpt_f1", "f2": "midpt_f2", "f3": "midpt_f3", "f4": "midpt_f4"})
    qrtrrow = get_row_nearest_time(df, qrtr, 'sec')
    qrtrdf = qrtrrow.rename(columns={"sec": "qrtr_sec", "f1": "qrtr_f1", "f2": "qrtr_f2", "f3": "qrtr_f3", "f4": "qrtr_f4"})
    thrqrtrrow = get_row_nearest_time(df, thrqrtr, 'sec')
    thrqrtrdf = thrqrtrrow.rename(columns={"sec": "thrqrtr_sec", "f1": "thrqrtr_f1", "f2": "thrqrtr_f2", "f3": "thrqrtr_f3", "f4": "thrqrtr_f4"})

    rowdf = pd.DataFrame([row])

    rowdffinal = rowdf.reindex(columns=['t1', 't2_ph', 'Phone', 'barename', 'Word', 'spkr', 'age_yrs', 'task']) # :=all rows, and these columns

    return pd.concat([midptdf, qrtrdf, thrqrtrdf, rowdffinal], axis=1)

In [ ]:
# run the function to match ifc meas and tg info
ifclist = []


for row in voweldf2.itertuples(): 
    ifclist.append(ifc_summary(row))
  

ifc_final = pd.concat(ifclist)

In [ ]:
ifc_final.head() # sanity check

In [ ]:
# remove F3 and F4 from lpc because we don't need it
ifc_final = ifc_final.drop(['midpt_f3', 'qrtr_f3', 'thrqrtr_f3', 'midpt_f3', 'qrtr_f3', 'thrqrtr_f3', 'midpt_f4', 'qrtr_f4', 'thrqrtr_f4'], axis=1)

## Merge results from three trackers

In [ ]:
repeats = ['Phone', 't2_ph', 'Word', 'spkr', 'age_yrs', 'task']

gettwo = pd.merge(    # merge ifc and ac dfs on filename and phone_t1
    ifc_final, 
    ac_final.drop(repeats, axis='columns'),  
    on=['barename', 't1'], 
    how='outer', 
    suffixes=['_ifc', '_ac']
) 

allthree = pd.merge(    # merge ifc, ac, and cov dfs on filename and phone_t1
    gettwo, 
    cov_final.drop(repeats, axis='columns'),  
    on=['barename', 't1'], 
    how='outer'
) 


In [ ]:
# sanity check - if every key had a match, should report FALSE
np.any(allthree.isnull())

## Calculate median formant meas for each point in vowel

In [ ]:
def get_median(row):
    
    # first clean up metadata
    tokeep = ['spkr', 'age_yrs', 'Phone', 'Word', 'barename', 'task', 't1', 't2_ph',
             'qrtr_sec', 'midpt_sec', 'thrqrtr_sec', 'qrtr_sec_ifc', 'midpt_sec_ifc', 'thrqrtr_sec_ifc',
             'qrtr_sec_ac', 'midpt_sec_ac', 'thrqrtr_sec_ac']
    
    
    rowdf = pd.DataFrame([row]).rei[:, tokeep]
    
    
    # now calculate median formant across trackers and formants
    f1_qrtr = pd.DataFrame([
        np.median(
            np.array(
                [row.qrtr_f1_ifc, row.qrtr_f1, row.qrtr_f1_ac]))], columns=['f1_qrtr_med'])
    f1_midpt = pd.DataFrame([
        np.median(
            np.array(
                [row.midpt_f1_ifc, row.midpt_f1, row.midpt_f1_ac]))], columns=['f1_midpt_med'])
    f1_thrqrtr = pd.DataFrame([
        np.median(
            np.array(
                [row.thrqrtr_f1_ifc, row.thrqrtr_f1, row.thrqrtr_f1_ac]))], columns=['f1_thrqrtr_med'])
    
    f2_qrtr = pd.DataFrame([
        np.median(
            np.array(
                [row.qrtr_f2_ifc, row.qrtr_f2, row.qrtr_f2_ac]))], columns=['f2_qrtr_med'])
    f2_midpt = pd.DataFrame([
        np.median(
            np.array(
                [row.midpt_f2_ifc, row.midpt_f2, row.midpt_f2_ac]))], columns=['f2_midpt_med'])
    f2_thrqrtr = pd.DataFrame(
        [np.median(
            np.array(
                [row.thrqrtr_f2_ifc, row.thrqrtr_f2, row.thrqrtr_f2_ac]))], columns=['f2_thrqrtr_med'])

    return pd.concat([f1_qrtr,  f1_midpt, f1_thrqrtr,
                          f2_qrtr, f2_midpt, f2_thrqrtr,
                          rowdf], axis=1)

In [ ]:
# now calculate median meas for each formant at 25, 50, and 75% of each vowel

medlist = []

for row in allthree.itertuples(): # loop through all rows in the df with ifc, cov, and ac measurements
    get_median(row)
    medlist.append(get_median(row))
  

medfinal = pd.concat(medlist)

In [ ]:
medfinal.head()

In [ ]:
# export data
medfinal.to_csv('med_formants_lpcorder_8.csv', index=False, header=True)

# Check for outliers
Plot some histograms to identify outliers on a by-participant and by-age group basis to determine if we should retrack formants with a different LPC order. 

In [ ]:
speaker = "AnaValeria" # specify the child

sub_df = medfinal.loc[(medfinal.spkr==speaker)]  # subset speakers and vowels.

ax2 = sns.distplot(sub_df["f1_qrtr_med"], hist = False, kde = True, kde_kws = {'shade': True, 'linewidth': 3})
sns.distplot(sub_df["f2_qrtr_med"], hist = False, kde = True, kde_kws = {'shade': True, 'linewidth': 3},ax=ax2)

ax2.set(xlabel='Formant frequency (Hz)', ylabel='Probability density')
ax2.text(350,0.001,"F1")
ax2.text(1200,0.0003,"F2")
ax2.text(3000,0.001,speaker,fontsize=22)


option to check by age

In [ ]:
type(medfinal.age_yrs)

In [ ]:
#medfinal.age_yrs = chr(medfinal.age_yrs)
for a in medfinal.age_yrs.unique():
        
    sub_df = medfinal.loc[(medfinal.age_yrs==a)]  # select age group

    plt.figure(a) # to separate the plots (remove if want overlaid)
    ax2 = sns.distplot(sub_df["f1_qrtr_med"], hist = False, kde = True, kde_kws = {'shade': True, 'linewidth': 3})
    sns.distplot(sub_df["f2_qrtr_med"], hist = False, kde = True, kde_kws = {'shade': True, 'linewidth': 3},ax=ax2)
    sns.distplot(sub_df["f3_qrtr_med"], hist = False, kde = True, kde_kws = {'shade': True, 'linewidth': 3},ax=ax2)

    ax2.set(xlabel='Formant frequency (Hz)', ylabel='Probability density')
    ax2.text(350,0.001,"F1")
    ax2.text(1200,0.0003,"F2")
    ax2.text(2300,0.0004,"F3")
    ax2.text(3000,0.001,a,fontsize=22)

# calculate each speaker's vocal tract length

In [ ]:
def get_vtl(d):
    deltaf = (np.mean(d.f1)/0.5 + np.mean(d.f2)/1.5 + np.mean(d.f3)/2.5 + np.mean(d.f4)/3.5)/4
    vtl = 34000/(2*deltaf)

    # Here is the Lammert & Nagarayan formula:  229 + 0.03F1 + 0.082*F2/3 + 0.124*F3/5 + 0.354*F4/7
    #phi = 229 + 0.030*np.mean(d.f1) + 0.02733*np.mean(d.f2) + 0.0248*np.mean(d.f3) + 0.05057*np.mean(d.f4)
    #vtl = 34000/(4*phi)
    #deltaf = phi*2
    
    return deltaf,vtl

In [ ]:
once we have vtl and speaking rate controlled for, what are the remaining differences 
between adults and kids? these are due to the morphology of the vt